In [21]:
import numpy as np
import os
import tempfile
import subprocess
from ultralytics import YOLO
import torch
import onnx2tf
import onnx
import onnx2keras
from onnx2keras import onnx_to_keras
from tqdm.auto import tqdm
import xml.etree.ElementTree as ET

import tensorflow as tf
from tensorflow import keras

import keras_cv
from keras_cv import bounding_box
from keras_cv import visualization

Using TensorFlow backend


# Helper Functions


In [3]:
def get_file_size(file_path):
    size = os.path.getsize(file_path)
    return size


def convert_bytes(size, unit=None):
    if unit == "KB":
        return print("File size: " + str(round(size / 1024, 3)) + " Kilobytes")
    elif unit == "MB":
        return print("File size: " + str(round(size / (1024 * 1024), 3)) + " Megabytes")
    else:
        return print("File size: " + str(size) + " bytes")


def c_style_hexdump(input, ouput, name):
    with open(input, "rb") as f:
        file = f.read()

    file = bytearray(file)
    _bytes = [f"0x{x:02x}" for x in file]
    file = ",".join(_bytes)

    with open(ouput, "w") as f:
        f.write("#pragma once\n")
        f.write("#include <stdalign.h>\n")
        f.write(f"alignas(16) const unsigned char {name}[] = {{{file}}};")

    return len(_bytes)


def build_header(output, names_with_sizes):
    with open(output, "w") as f:
        f.write('#pragma once\n#ifdef __cplusplus\nextern "C"\n{\n#endif\n')
        f.write("#include <stdalign.h>\n\n")
        for name, size in names_with_sizes:
            f.write(f"alignas(16) extern const unsigned char {name}[{size}];\n")
        f.write("\n#ifdef __cplusplus\n}\n#endif\n")


def evaluate_model(interpreter, x_test, y_test):
    input_index = interpreter.get_input_details()[0]["index"]
    output_index = interpreter.get_output_details()[0]["index"]

    # Run predictions on every image in the "test" dataset.
    prediction_digits = []
    for i, test_image in enumerate(x_test):
        # Pre-processing: add batch dimension and convert to float32 to match with
        # the model's input data format.
        test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
        interpreter.set_tensor(input_index, test_image)

        # Run inference.
        interpreter.invoke()

        # Post-processing: remove batch dimension and find the digit with highest
        # probability.
        output = interpreter.tensor(output_index)
        digit = np.argmax(output()[0])
        prediction_digits.append(digit)

    print("\n")
    # Compare prediction results with ground truth labels to calculate accuracy.
    prediction_digits = np.array(prediction_digits)
    accuracy = (prediction_digits == y_test).mean()
    return accuracy

# Load the Model


In [10]:
model = YOLO("yolov8n.pt")
results = model.val(data="coco128.yaml")

Ultralytics YOLOv8.2.10 🚀 Python-3.10.12 torch-2.1.2+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Ti, 8192MiB)
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs


val: Scanning /home/eduard/Github/datasets/coco128/labels/train2017.cache... 126 images, 2 backgrounds, 0 corrupt: 100%|██████████| 128/128 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 8/8 [00:02<00:00,  3.16it/s]


                   all        128        929       0.64      0.537      0.605      0.446
                person        128        254      0.796      0.677      0.765      0.538
               bicycle        128          6      0.514      0.333      0.315      0.264
                   car        128         46      0.813      0.217      0.273      0.168
            motorcycle        128          5      0.687      0.887      0.898      0.685
              airplane        128          6      0.819      0.833      0.927      0.675
                   bus        128          7      0.492      0.714      0.728      0.671
                 train        128          3      0.534      0.667      0.706      0.604
                 truck        128         12          1      0.332      0.473      0.297
                  boat        128          6      0.226      0.167      0.316      0.135
         traffic light        128         14      0.734      0.201      0.202      0.139
             stop sig

In [11]:
model.export(format="tflite", imgsz=640, int8=True, data="coco128.yaml")

Ultralytics YOLOv8.2.10 🚀 Python-3.10.12 torch-2.1.2+cu121 CPU (11th Gen Intel Core(TM) i5-11400F 2.60GHz)

PyTorch: starting from 'yolov8n.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (6.2 MB)

TensorFlow SavedModel: starting export with tensorflow 2.13.1...

ONNX: starting export with onnx 1.16.0 opset 17...
ONNX: simplifying with onnxsim 0.4.36...
ONNX: export success ✅ 0.9s, saved as 'yolov8n.onnx' (12.3 MB)
TensorFlow SavedModel: collecting INT8 calibration images from 'data=coco128.yaml'


Scanning /home/eduard/Github/datasets/coco128/labels/train2017.cache... 126 images, 2 backgrounds, 0 corrupt: 100%|██████████| 128/128 [00:00<?, ?it/s]

TensorFlow SavedModel: WARNING ⚠️ >300 images recommended for INT8 calibration, found 128 images.
TensorFlow SavedModel: export failure ❌ 0.9s: permute(sparse_coo): number of dimensions in the tensor input does not match the length of the desired ordering of dimensions i.e. input.dim() = 4 is not equal to len(dims) = 3


RuntimeError: permute(sparse_coo): number of dimensions in the tensor input does not match the length of the desired ordering of dimensions i.e. input.dim() = 4 is not equal to len(dims) = 3

In [16]:
model.export(format="tflite", imgsz=640, data="coco128.yaml")

Ultralytics YOLOv8.2.10 🚀 Python-3.10.12 torch-2.1.2+cu121 CPU (11th Gen Intel Core(TM) i5-11400F 2.60GHz)

PyTorch: starting from 'yolov8n.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (6.2 MB)

TensorFlow SavedModel: starting export with tensorflow 2.13.1...

ONNX: starting export with onnx 1.16.0 opset 17...
ONNX: simplifying with onnxsim 0.4.36...
ONNX: export success ✅ 0.7s, saved as 'yolov8n.onnx' (12.3 MB)
TensorFlow SavedModel: starting TFLite export with onnx2tf 1.17.5...


2024-05-09 23:46:01.784311: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-09 23:46:01.784373: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2024-05-09 23:46:01.784513: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2024-05-09 23:46:01.784982: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-05-09 23:46:01.785039: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/

TensorFlow SavedModel: export success ✅ 59.6s, saved as 'yolov8n_saved_model' (30.9 MB)

TensorFlow Lite: starting export with tensorflow 2.13.1...
TensorFlow Lite: export success ✅ 0.0s, saved as 'yolov8n_saved_model/yolov8n_float32.tflite' (12.3 MB)

Export complete (61.8s)
Results saved to /home/eduard/Github/x-heep-femu-tflite-sdk/sw/riscv/apps/tflite_yolo
Predict:         yolo predict task=detect model=yolov8n_saved_model/yolov8n_float32.tflite imgsz=640  
Validate:        yolo val task=detect model=yolov8n_saved_model/yolov8n_float32.tflite imgsz=640 data=coco.yaml  
Visualize:       https://netron.app


'yolov8n_saved_model/yolov8n_float32.tflite'

In [14]:
converter = tf.lite.TFLiteConverter.from_saved_model("yolov8n_saved_model")
file = converter.convert()

_, temp = tempfile.mkstemp(".tflite")

with open(temp, "wb") as f:
    f.write(file)

convert_bytes(get_file_size(temp), "MB")

ValueError: Only support at least one signature key.

In [ ]:
# # validate on COCO dataset FP32
# tflite_yolo = YOLO("yolov8n_saved_model/yolov8n_float32.tflite")
# tflite_yolo.val(data="coco.yaml", separate_outputs=True)

# # validate on COCO dataset INT8
# tflite_yolo = YOLO("yolov8n_saved_model/yolov8n_full_integer_quant.tflite")
# tflite_yolo.val(data="coco.yaml", separate_outputs=True)

## Convert onnx to keras


In [19]:
onnx_model = onnx.load("yolov8n.onnx")
k_model = onnx2keras.onnx_to_keras(onnx_model, ["input_0"])

AttributeError: Current node is not in weights / model inputs / layers.

## Fit a YOLOv8 model with keras directly

https://keras.io/examples/vision/yolov8/


In [36]:
backbone = keras_cv.models.YOLOV8Backbone.from_preset("yolo_v8_xs_backbone_coco")
# https://github.com/keras-team/keras-cv/issues/1886

ValueError: The channel dimension of the inputs should be defined. The input_shape received is (None, None, None, 5), where axis -3 (0-based) is the channel dimension, which found to be `None`.